In [0]:
%run "/Workspace/Formula1Project/includes/configuration"

In [0]:
%run "/Workspace/Formula1Project/includes/common_functions"

In [0]:
dbutils.widgets.text("p_file_date", "")
v_file_date = dbutils.widgets.get("p_file_date")
print(v_file_date)

In [0]:
race_results_list = spark.read.format("delta").load(f"{presentation_path}/race_results", header=True).filter(f"results_file_date = '{v_file_date}'").select("race_year").distinct().collect()

In [0]:
from pyspark.sql.functions import col
race_years = []
for race_year in race_results_list:
    race_years.append(race_year.race_year)
race_results_df = spark.read.format("delta").load(f"{presentation_path}/race_results", header=True).filter(col("race_year").isin(race_years))

In [0]:
from pyspark.sql.functions import sum, when, count, col

results_grouped_df = race_results_df.groupBy(
     "race_year","team"
).agg(
    sum("points").alias("total_points"), 
    count(when(col("position") == 1, True)).alias("wins")
)

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import rank, desc

spec = Window.partitionBy("race_year").orderBy(desc("total_points"),desc("wins"))
display(results_grouped_df.withColumn("rank", rank().over(spec)))

In [0]:
merge_condition = "tgt.race_year = src.race_year and tgt.team = src.team"
merge_data(results_grouped_df, presentation_path, "f1_presentation", "constructor_standings", "race_year", merge_condition)

In [0]:
#overwrite_partition(results_grouped_df,'race_year','f1_presentation','constructor_standings')

In [0]:
#results_grouped_df.write.mode("overwrite").format("parquet").saveAsTable("f1_presentation.constructor_standings")